In [27]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [28]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"

In [29]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [44]:
import os
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [31]:
llm_model = "gpt-3.5-turbo"

In [32]:
from langchain.text_splitter import CharacterTextSplitter

In [36]:
import os
from langchain.schema import Document
from langchain_text_splitters.character import CharacterTextSplitter  # Import the specific splitter

directory_path = '/content/drive/My Drive/HowToPlayWeiss'
chunk_size = 1000
chunk_overlap = 200
all_chunks = []

for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):  # Process only .txt files
        file_path = os.path.join(directory_path, filename)

        # Load the text file
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()

        # Split the text into chunks manually
        start = 0
        while start < len(data):
            end = min(start + chunk_size, len(data))
            chunk = data[start:end]
            all_chunks.append(chunk)
            start += (chunk_size - chunk_overlap)  # Move start pointer forward by chunk_size - chunk_overlap

print(f"Total chunks: {len(all_chunks)}")


Total chunks: 694


In [39]:
embeddings = OpenAIEmbeddings()
all_documents = [Document(page_content=chunk) for chunk in all_chunks]
vectorstore = FAISS.from_documents(all_documents, embedding=embeddings)

In [40]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [41]:
query = "When is it good to encore a chatacter"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'Encoring a character can be beneficial in several situations in a game. \n\n1. If a character has a powerful ability or effect that can significantly influence the game, it might be beneficial to Encore them to keep them in play.\n\n2. If a character is in a key position on the field (like the center stage), Encoring them can maintain your field setup and potentially protect weaker characters.\n\n3. If a character has high power or a unique trait that makes them difficult for your opponent to deal with, Encoring them can keep the pressure on your opponent.\n\n4. If a character has an Encore ability that triggers a beneficial effect when they are Encored (as per the text "When (player) use this card’s Encore" referring to when a specified character is placed on the stage from the waiting room by the effects of Encore), it might be beneficial to Encore them to use this effect.\n\nRemember that the decision to Encore a character often involves a cost (like paying 3 cards from your Stock 

In [ ]:
query = "What is the contact information?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

In [ ]:
query = "What are the main activities of ScaleX Innovation. Write is as three bullet points."
result = conversation_chain({"question": query})
answer = result["answer"]
answer